# Import

In [1]:
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch


  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-hj44bs3s
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-hj44bs3s
     |████████████████████████████████| 58 kB 1.7 MB/s 
     |████████████████████████████████| 244 kB 4.2 MB/s 
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.1.3-py3-none-any.whl size=83164 sha256=090f3d048e48f0c48115516ac95dacb4abc0ebc5b7b40fbd6ef4084c7c7cac83
  Stored in directory: /tmp/pip-ephem-wheel-cache-uedqnrvf/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=893b2c0568ad67279d43f2fefa0be8923a71d3b4b069105279ddc9035715a2de
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-no

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage import io
import matplotlib.pyplot as plt


import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, Sampler, random_split
import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer
import torchmetrics
import segmentation_models_pytorch as smp

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


In [3]:
import warnings
warnings.filterwarnings("ignore")

# Setup

In [4]:
def get_change_pct(row):
    """Helper to get change mask percentage"""
    ch = np.abs(np.divide(io.imread(DATA_DIR + row['mask_path']),255))
    shp = ch.shape[0] * ch.shape[1]
    return np.sum(ch)/shp

In [5]:
grayscale = False

if grayscale == True:
    IN_CHANNELS = 2
else:
    IN_CHANNELS = 6

RANDOM_SEED = 42
BATCH_SIZE = 64
DATA_DIR = "../input/spacenet-7-change-detection-chips-and-masks/chip_dataset/chip_dataset/change_detection/"
ANNOTATIONS = "../input/satellite-models/df.csv"
CHECKPOINT = "../input/satellite-models/epoch20-val_IoU0.51-val_loss1.51.ckpt"

annotations = pd.read_csv(ANNOTATIONS)
# annotations = annotations[annotations.target == 1] # manually excluding blank chips and avoiding possibility to use sampler
# # annotations['ch_pct'] = annotations.progress_apply(lambda x: get_change_pct(x), axis = 1) 
# annotations = annotations[annotations.ch_pct > 0.02] #excluding irrelevant chips with critically small change percentage (glitches mostly)
# annotations.reset_index(inplace = True)

# aoi = annotations['im_name'].unique()
# train_aoi = aoi[:58]
# valid_aoi = aoi[-2:-1]
# test_aoi = aoi[-1:]

# def choose_aoi(df, names):
#     mask = df['im_name'].map(lambda x: x in names)
#     return df[mask].reset_index(drop=True)

# df_dict = {'train' : choose_aoi(annotations, train_aoi),
#           'test' : choose_aoi(annotations, test_aoi),
#           'valid' : choose_aoi(annotations, valid_aoi)}

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

    
df_dict = {'train' : pd.read_csv('../input/satellite-models/train_csv .csv'),
           'valid' : pd.read_csv('../input/satellite-models/valid_csv.csv')}


In [6]:
len(annotations)

187705

In [7]:
import ast
import requests
file = open("../input/logging-utils/credentials.txt", "r")
contents = file.read()
token = ast.literal_eval(contents)
file.close()

def telegram_bot_sendtext(bot_message):
    send_text = 'https://api.telegram.org/bot' + token['bot_token'] + '/sendMessage?chat_id=' + token['bot_chatID'] + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

def telegram_send_file (file_address):
    url = f'https://api.telegram.org/bot' + token['bot_token'] + '/sendVoice'
    #response = requests.post(url, data=data)
    post_data = {'chat_id': token['bot_chatID']}
    with open(file_address, 'r+b') as infile:
        post_file = {'document': infile}
        r = requests.post(f'https://api.telegram.org/bot' +token['bot_token'] + '/sendDocument', data=post_data, files=post_file)
        print(r.text)


test = telegram_bot_sendtext("Testing Telegram bot")
print(test)

def log_traceback(ex, ex_traceback=None):
    if ex_traceback is None:
        ex_traceback = ex.__traceback__
    tb_lines = [ line.rstrip('\n') for line in
                 traceback.format_exception(ex.__class__, ex, ex_traceback)]
    return tb_lines

{'ok': True, 'result': {'message_id': 13536, 'from': {'id': 1759463282, 'is_bot': True, 'first_name': 'my_project_logger', 'username': 'satellite_study_bot'}, 'chat': {'id': 202015929, 'first_name': 'R', 'last_name': 'I', 'username': 'runRudy', 'type': 'private'}, 'date': 1624771375, 'text': 'Testing Telegram bot'}}


# Lightning Datamodule

In [8]:
class CD_Dataset(Dataset):
    """Dataset class
    Args:
        root_folder: Path object, root directory of picture dataset
        csv: pandas.DataFrame, untidy df with all data relationships
        aug: albumentations dictionary
        preproc: callable, preprocessing function related to specific encoder
        grayscale: boolean, preprocessing condition to grayscale colored rasters
    Return:
        image, mask tensors"""
    
    def __init__(self, df, root_folder, grayscale = False, predict = False):
        self.root_folder = root_folder
        self.csv = df
        self.grayscale = grayscale
        self.predict = predict
    
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        chip_path = self.root_folder + self.csv.loc[idx,'chip_path']
        # read chip into numpy array
        chip = io.imread(self.root_folder + self.csv.loc[idx,'chip_path']).astype('float32')
        # get target for corresponding chip
        mask = np.abs(np.divide(io.imread(self.root_folder + self.csv.loc[idx,'mask_path']),255)).astype('float32')
        if self.grayscale == True:
            gray1 = np.dot(chip[:,:,0:3], [0.2989, 0.5870, 0.1140])
            gray2 = np.dot(chip[:,:,3:], [0.2989, 0.5870, 0.1140])
            chip = np.divide(np.stack((gray1, gray2),axis = 2),255).astype('float32')
            del(gray1,gray2)
        image = torch.Tensor(np.moveaxis(chip, 2, 0))
        mask = torch.Tensor(mask).unsqueeze(0)
        del(chip)
        if self.predict == False:
            return image, mask
        else: 
            return image
class BalancedSampler(Sampler):
    """Balancer for torch.DataLoader to adjust chips loading"""
    
    def __init__(self, dataset, percentage = 0.5):
        """
        dataset: custom torch dataset
        percentage: float number between 0 and 1, percentage of change containing pictures in batch
        """
        assert 0 <= percentage <= 1,'percentage must be a value between 0 and 1'
        
        self.dataset = dataset
        self.pct = percentage
        self.len_ = len(dataset)
    
    def __len__(self):
        return self.len_
    def quart_chk(self,val):
        if 0.03 <= val <= 0.08:
            return True
        else: 
            False
    
    def __iter__(self):
        # get indices for chips containing change and blank ones
        inside = np.where(self.dataset.csv['ch_pct'] < 0.08)[0]
        outside = np.where(self.dataset.csv['ch_pct'] > 0.08)[0]
        # randomly sample from the incides of each class according to percentage value
        inside = np.random.choice(inside,int(self.len_ * self.pct), replace=True)
        outside = np.random.choice(outside,int(self.len_ * (1 - self.pct))+1, replace=False)
        # stack and shuffle of sampled indices
        all_idxs = np.hstack([inside,outside])
        np.random.shuffle(all_idxs)
        return iter(all_idxs)

# Lightning data module

In [9]:
import albumentations as alb
# let's get few variants of torchvision transformations
trfs = alb.Compose([
                alb.HorizontalFlip(),
                alb.VerticalFlip(),
    ])


In [10]:
class CD_DataModule(pl.LightningDataModule):

    def __init__(self, transform, batch_size: int = BATCH_SIZE, grayscale: bool = False):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transform
        self.state_dict = {}
        if self.transform == None:
            self.transform = alb.Compose([alb.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            ])
        self.dims = (3, 64, 64)
    
    def setup(self, stage):
            self.train = CD_Dataset(root_folder = DATA_DIR,df = df_dict['train'])
            self.val = CD_Dataset(root_folder = DATA_DIR,df = df_dict['valid'])
            self.test = CD_Dataset(root_folder = DATA_DIR,df = df_dict['valid'])
            self.predict = CD_Dataset(root_folder = DATA_DIR,df = df_dict['valid'], predict = True)
        
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, num_workers = 4)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers = 4)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers = 4)
    
    def predict_dataloader(self):
        return DataLoader(self.predict, batch_size=self.batch_size, num_workers = 4)

    def teardown(self, stage = None):
        # Used to clean-up when the run is finished
        ...


In [11]:
SpaceNet7 = CD_DataModule(transform = trfs, batch_size = BATCH_SIZE, grayscale = False)

# Segmentation model

In [12]:
checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor = 'val_IoU', mode = 'max', save_last = True,
...     dirpath='./',
...     filename='{epoch:02d}-{val_IoU:.2f}-{val_loss:.2f}'
... )
prog_bar_cb = pl.callbacks.ProgressBar()
early_stopping = pl.callbacks.EarlyStopping(monitor = 'val_IoU',mode = 'max', patience = 5)

callbacks = [checkpoint_callback, prog_bar_cb, early_stopping]

In [13]:
import gc
gc.collect()

60

In [14]:
 class LitModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = smp.Unet('vgg16', encoder_weights = 'imagenet', in_channels=IN_CHANNELS , activation='sigmoid')
        self.loss = smp.losses.DiceLoss(mode='binary')
        self.metrics = torchmetrics.IoU(num_classes = 2)
        self.batch_size = 64
        self.lr = 0.005

    def forward(self, x):
        x = self.model(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr = self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1)
        return {
            'optimizer':optimizer,
            'scheduler': scheduler, # The LR scheduler instance (required)
            # The unit of the scheduler's step size, could also be 'step'
            'interval': 'epoch',
            'frequency': 1, # The frequency of the scheduler
            'monitor': 'val_IoU', # Metric for `ReduceLROnPlateau` to monitor
            'strict': True, # Whether to crash the training if `monitor` is not found
            'name': None, # Custom name for `LearningRateMonitor` to use
            }
    
    def evaluate(self, batch, phase):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        if phase == 'test' or 'val':
            metric = self.metrics(y_hat, y.int())
            self.log(f'{phase}_IoU', metric, prog_bar = True)
            self.log(f'{phase}_loss', loss, prog_bar = True)
        return loss
    
    def training_step(self, batch, batch_idx):
        loss = self.evaluate(batch,'train')
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = self.evaluate(batch, 'val')
        return loss
    
    def test_step(self, batch, batch_idx):
        loss = self.evaluate(batch, 'test')
        return loss

In [15]:
net = LitModel()
# net.load_from_checkpoint('../input/satellite-models/last (1).ckpt')
# sanity_trainer = Trainer(tpu_cores = 2, overfit_batches=0.01)
# sanity_trainer.fit(net, SpaceNet7)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [16]:
lr_finder = Trainer(gpus=-1,auto_lr_find = True)
lr_finder.tune(net, SpaceNet7)

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
net.lr
telegram_bot_sendtext(f'{net.lr}')

{'ok': True,
 'result': {'message_id': 13537,
  'from': {'id': 1759463282,
   'is_bot': True,
   'first_name': 'my_project_logger',
   'username': 'satellite_study_bot'},
  'chat': {'id': 202015929,
   'first_name': 'R',
   'last_name': 'I',
   'username': 'runRudy',
   'type': 'private'},
  'date': 1624771463,
  'text': '0.00478630092322638'}}

In [18]:
trainer = Trainer(callbacks = callbacks, gpus=-1, check_val_every_n_epoch = 1, max_epochs = 100, precision = 16) # let's use half precision to increase gpu usage and possibly performance
trainer.fit(net, SpaceNet7)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

# Evaluation

In [19]:
res = trainer.test(net, datamodule=SpaceNet7)
telegram_bot_sendtext(f'{res}') 

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_IoU': 0.4913859963417053, 'test_loss': nan}
--------------------------------------------------------------------------------


{'ok': True,
 'result': {'message_id': 13538,
  'from': {'id': 1759463282,
   'is_bot': True,
   'first_name': 'my_project_logger',
   'username': 'satellite_study_bot'},
  'chat': {'id': 202015929,
   'first_name': 'R',
   'last_name': 'I',
   'username': 'runRudy',
   'type': 'private'},
  'date': 1624778882,
  'text': "{'test_IoU': 0.4913859963417053, 'test_loss': nan}"}}

In [20]:
batch = next(iter(SpaceNet7.predict_dataloader()))

In [21]:
len(batch)

64

In [22]:
# n = np.random.randint(63)
# im1,im2 = batch[0][n],batch[1][n]
# y_hat = net((im1.unsqueeze(0),im2.unsqueeze(0)))

# _,axes = plt.subplots(1,3)
# axes[0].imshow(np.moveaxis(im1.cpu().numpy().astype('uint8'),0,2))
# axes[1].imshow(np.moveaxis(im2.cpu().numpy().astype('uint8'),0,2))
# axes[2].imshow(y_hat.cpu().squeeze().detach().numpy().astype('float32'))



In [23]:
# print(type(im1),im1.shape)

# Saving

In [24]:
model_str = './model.ckpt'
torch.save(net, model_str)

Inf

In [25]:
trainer.save_checkpoint("example.ckpt")

# Sandbox

To try to understand why is our net is overfitting, let's get a closer look to the dataset part descriptions.

In [26]:
df_dict['train'].describe()

,target,year1,month1,year2,month2,x,y,is_blank,n_change_pix,month_diff,ch_pct
count,162988.0,162988.000000,162988.000000,162988.000000,162988.000000,162988.000000,162988.000000,0.0,162988.000000,162988.000000,162988.000000
mean,1.0,2018.473980,6.305845,2018.530616,6.246405,508.129089,522.234079,NaN,182.004270,12.041512,0.044435
std,0.0,0.672385,3.528176,0.627730,3.635042,282.477829,273.128484,NaN,203.692557,5.748464,0.049730
min,1.0,2017.000000,1.000000,2017.000000,1.000000,0.000000,0.000000,NaN,41.000000,1.000000,0.010010
25%,1.0,2018.000000,3.000000,2018.000000,3.000000,320.000000,320.000000,NaN,61.000000,8.000000,0.014893
50%,1.0,2018.000000,6.000000,2019.000000,6.000000,512.000000,512.000000,NaN,102.000000,12.000000,0.024902
75%,1.0,2019.000000,9.000000,2019.000000,9.000000,704.000000,768.000000,NaN,227.000000,16.000000,0.055420
max,1.0,2020.000000,12.000000,2020.000000,12.000000,960.000000,960.000000,NaN,3130.000000,26.000000,0.764160


In [27]:
df_dict['valid'].describe()

,target,year1,month1,year2,month2,x,y,is_blank,n_change_pix,month_diff,ch_pct
count,57358.0,57358.000000,57358.000000,57358.000000,57358.000000,57358.000000,57358.000000,0.0,57358.000000,57358.000000,57358.000000
mean,1.0,2018.571202,6.764409,2018.651104,5.691255,486.733847,402.944036,NaN,70.565762,11.053593,0.017228
std,0.0,0.558341,3.611558,0.618957,3.639952,279.835664,251.467948,NaN,130.715306,5.689270,0.031913
min,1.0,2018.000000,1.000000,2018.000000,1.000000,0.000000,0.000000,NaN,1.000000,1.000000,0.000244
25%,1.0,2018.000000,4.000000,2018.000000,2.000000,256.000000,192.000000,NaN,15.000000,6.000000,0.003662
50%,1.0,2019.000000,7.000000,2019.000000,5.000000,512.000000,384.000000,NaN,33.000000,11.000000,0.008057
75%,1.0,2019.000000,10.000000,2019.000000,9.000000,704.000000,640.000000,NaN,74.000000,15.000000,0.018066
max,1.0,2020.000000,12.000000,2020.000000,12.000000,960.000000,960.000000,NaN,1961.000000,24.000000,0.478760


In [28]:
df_dict['test'].describe()

KeyError: 'test'